Instalamos librerias, y subimos el archivo data_final2.csv

In [1]:
!pip install num2words
!pip install unzip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=038f95d124442b0011b6b551cb2306a49259acbf621882cc9304c1b9c97311c7
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
  Preparing metadata (setup.py) ... done
  Created wheel for unzip: filename=unzip-1.0.0-py3-none-any.whl size=1279 sha256=90a81c4ecda67183008a00b19ed28904094347511f3e70a0a27ae1de3cab01e0
  Stored in directory: /root/.cache/pip/wheels/80/dc/7a/f8af45bc239e7933509183f038ea8d46f3610aab82b35369f4
Successfully built unzip


In [2]:
import re  # Para Preprocesamiento
import pandas as pd
from time import time  # Tiempo de las operaciones
from collections import defaultdict  # Para Frecuencias de palabras

import spacy  # Para prepocesamiento
import logging  # Configuración de loggings para monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
from gensim.models.phrases import Phrases, Phraser

Cargo los datos con el df del primer cuaderno

In [3]:
df = pd.read_csv('data_final2.csv')
df.shape

(5000, 8)

Previsualizo

In [4]:
df.head()

,overall,verified,reviewTime,reviewerID,reviewerName,reviewText,summary,sentiment_label
0,1,True,"02 10, 2017",A2CRWP2FB4TKMT,Joanne B. Martin,bought as Xmas present for my collage girl. Th...,disappointing,1
1,1,True,"06 6, 2014",A17NCZFPT37WSH,tr maushardt,Although I used the gift card for other events...,THIS CARD EXPIRED BEFORE RECIPIENT COULD USE IT!,1
2,1,True,"05 30, 2016",A2BPII513TLOH3,Amazon Customer,It came Within a couple min.,One Star,1
3,1,True,"01 12, 2018",A2QVRF04UGWZGJ,Marian Shirley,Card won't be accepted when trying to use it f...,What can we do?,1
4,1,True,"12 28, 2017",A1BBUPYYRCGE3S,Danielle,Amazon kept screwing with my codes and I lost ...,Amazon screwed up. Just give cash. This isn't ...,1


Preparo el Pipeline para realizar el preprocesado del texto a través de NLTK

In [5]:
import pandas as pd
import unicodedata
from num2words import num2words
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


# Creo los objetos para el procesamiento
tokenizer = RegexpTokenizer(r'\w+')
sw_list = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def nltk_cleaner(reviewText):
    if isinstance(reviewText, str):
        clean_text = list()

        # Elimino acentos y signos
        reviewText = unicodedata.normalize('NFKD', reviewText).encode('ascii', 'ignore').decode('utf-8', 'ignore')

        # Separo palabras eliminando signos de puntuación
        for word in tokenizer.tokenize(reviewText):

            # Elimino stop words
            if word not in sw_list:

                # Elimino espacios sobrantes, convierto a minúsculas y lematizo
                clean_word = lemmatizer.lemmatize(word).lower().strip()

                # Convierto los dígitos a palabras
                if clean_word.isdigit():
                    clean_word = num2words(clean_word, lang='en')

                clean_text.append(clean_word)

        return ' '.join(clean_text)
    else:
        return ''

# Aplico la función a la columna 'reviewText' y creo una nueva columna que llamo 'texto_limpio'
df['texto_limpio'] = df['reviewText'].apply(nltk_cleaner)

# Imprimo los primeros registros para verificar el resultado
print(df.head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


   overall  verified   reviewTime      reviewerID      reviewerName  \
0        1      True  02 10, 2017  A2CRWP2FB4TKMT  Joanne B. Martin   
1        1      True   06 6, 2014  A17NCZFPT37WSH      tr maushardt   
2        1      True  05 30, 2016  A2BPII513TLOH3   Amazon Customer   
3        1      True  01 12, 2018  A2QVRF04UGWZGJ    Marian Shirley   
4        1      True  12 28, 2017  A1BBUPYYRCGE3S          Danielle   

                                          reviewText  \
0  bought as Xmas present for my collage girl. Th...   
1  Although I used the gift card for other events...   
2                       It came Within a couple min.   
3  Card won't be accepted when trying to use it f...   
4  Amazon kept screwing with my codes and I lost ...   

                                             summary  sentiment_label  \
0                                      disappointing                1   
1   THIS CARD EXPIRED BEFORE RECIPIENT COULD USE IT!                1   
2                

In [6]:
df.head()

,overall,verified,reviewTime,reviewerID,reviewerName,reviewText,summary,sentiment_label,texto_limpio
0,1,True,"02 10, 2017",A2CRWP2FB4TKMT,Joanne B. Martin,bought as Xmas present for my collage girl. Th...,disappointing,1,bought xmas present collage girl they einstein...
1,1,True,"06 6, 2014",A17NCZFPT37WSH,tr maushardt,Although I used the gift card for other events...,THIS CARD EXPIRED BEFORE RECIPIENT COULD USE IT!,1,although i used gift card event this card coul...
2,1,True,"05 30, 2016",A2BPII513TLOH3,Amazon Customer,It came Within a couple min.,One Star,1,it came within couple min
3,1,True,"01 12, 2018",A2QVRF04UGWZGJ,Marian Shirley,Card won't be accepted when trying to use it f...,What can we do?,1,card accepted trying use order
4,1,True,"12 28, 2017",A1BBUPYYRCGE3S,Danielle,Amazon kept screwing with my codes and I lost ...,Amazon screwed up. Just give cash. This isn't ...,1,amazon kept screwing code i lost family christ...


Como conclusión, he decidido usar este pipeline porque creo que se ajusta bien al texto para eliminar el ruido y limpiar, tanto por la eliminación de puntuación y acentos, por la eliminación de espacios innecesarios y de mayuscular. Por último el pasar los digitos a texto la lemmatización nos ayuda a simplificar las palabras y esto facilitará su análisis y comparación en la siguiente etapa, con el entrenamiento del modelo.

Previsualizado el resultado, lo guardo

In [7]:
df.to_csv('data_prepocess.csv', index=False)